In [ ]:
import os
import boto3

testingMode = False

endpoint = "XXX" if testingMode else os.environ.get('AWS_S3_ENDPOINT')
bucket_name = "XXX" if testingMode else os.environ.get('AWS_S3_BUCKET')
key_id = "XXX" if testingMode else os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = "XXX" if testingMode else os.environ.get('AWS_SECRET_ACCESS_KEY')
session = boto3.session.Session(aws_access_key_id=key_id, aws_secret_access_key=secret_key)
s3_client = boto3.client('s3',endpoint_url=endpoint,aws_access_key_id=key_id, aws_secret_access_key=secret_key)
session = boto3.session.Session(aws_access_key_id=key_id, aws_secret_access_key=secret_key)
s3_client = boto3.client('s3',endpoint_url=endpoint,aws_access_key_id=key_id, aws_secret_access_key=secret_key)

def download_s3_folder(s3_folder, local_dir):
    s3 =  boto3.client('s3',endpoint_url=endpoint,aws_access_key_id=key_id, aws_secret_access_key=secret_key)
    # List all objects within the specified S3 folder
    objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_folder)['Contents']
    
    for obj in objects:
        # Extract the file path from the object key
        s3_key = obj['Key']
        
        # Create the local file path
        relative_path = os.path.relpath(s3_key, s3_folder)
        local_file_path = os.path.join(local_dir, relative_path)
        
        # Ensure local directory exists
        local_dir_path = os.path.dirname(local_file_path)
        if not os.path.exists(local_dir_path):
            os.makedirs(local_dir_path)
        
        # Download the file from S3
        s3.download_file(bucket_name, s3_key, local_file_path)
        print(f"Downloaded {s3_key} to {local_file_path}")
    
def upload_s3_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                s3_key = os.path.relpath(file_path, folder_path)
                try:
                    s3_client.upload_file(file_path, bucket_name, s3_key)
                    print(f'Successfully uploaded {file_path} to s3://{bucket_name}/{s3_key}')
                except FileNotFoundError:
                    print(f'The file {file_path} was not found')
                except NoCredentialsError:
                    print('Credentials not available')
                except PartialCredentialsError:
                    print('Incomplete credentials')

In [ ]:
# Download Pipeline Training Args
import json

download_s3_folder("PipelineArgs","PipelineArgs")

with open("PipelineArgs/PipelineArgs.json", "r") as file:
    pipelineArgs = json.load(file)

required_keys = ["MODEL_NAME", "HF_MODEL_REPO_ID", "HF_DATASET_ID","SYSTEM_INST"]

# Check if the required keys are present in the JSON data
if all(key in pipelineArgs for key in required_keys):
    print("The JSON file contains all the required keys.")
else:
    missing_keys = [key for key in required_keys if key not in pipelineArgs]
    print(f"The JSON file is missing the following keys: {', '.join(missing_keys)}")

print("Pipeline Arguments downloaded and verified")
print(pipelineArgs)

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git llamacpp

In [ ]:
!pip install peft torch==2.3.1 transformers
!pip install -r llamacpp/requirements.txt

In [ ]:
download_s3_folder(f"""{pipelineArgs["MODEL_NAME"]}/{pipelineArgs["HF_MODEL_REPO_ID"].split('/')[1]}""","Models")

In [ ]:
import os
from peft import PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer)

modelName = os.path.abspath("Models/BaseModel")
finetuned_model_id = os.path.abspath("Models/TrainedLoRA")
model = AutoModelForCausalLM.from_pretrained(
    modelName,
    device_map = 'auto',
    token = False,
)
peft_model = PeftModel.from_pretrained(
    model,
    finetuned_model_id,
    from_transformers=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained( modelName )
model = peft_model.merge_and_unload()
model.save_pretrained( "merged" )
tokenizer.save_pretrained( "merged" )



In [ ]:
!python llamacpp/convert_hf_to_gguf.py merged --outfile trainedModel.gguf --outtype q8_0

In [ ]:
dirpath = f"""Upload/{pipelineArgs["MODEL_NAME"]}/{pipelineArgs["HF_MODEL_REPO_ID"].split('/')[1]}/ollama"""
os.makedirs(dirpath,exist_ok=True)
os.rename("trainedModel.gguf", f"""{dirpath}/{pipelineArgs["MODEL_NAME"]}.gguf""" )

In [ ]:
config = f"""FROM /mnt/models/{pipelineArgs["MODEL_NAME"]}.gguf"""+"""
PARAMETER temperature 0.7
PARAMETER stop "<|im_start|>"
PARAMETER stop "<|im_end|>"
TEMPLATE \"""
<|im_start|>system
{{ .System }}<|im_end|>
<|im_start|>user
{{ .Prompt }}<|im_end|>
<|im_start|>assistant
\"""
""" + f"""SYSTEM \"""{pipelineArgs["SYSTEM_INST"]}""\""""

with open(f"{dirPath}/Modelfile", "w") as mf:
    mf.write(config)

In [ ]:
upload_s3_folder("Upload")